In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/predict-potential-spammers-on-fiverr/sample_submission.csv
/kaggle/input/predict-potential-spammers-on-fiverr/train.csv
/kaggle/input/predict-potential-spammers-on-fiverr/test.csv


In [2]:
from sklearn import metrics
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import xgboost as xgb


In [3]:
train = pd.read_csv('/kaggle/input/predict-potential-spammers-on-fiverr/train.csv')
test = pd.read_csv('/kaggle/input/predict-potential-spammers-on-fiverr/test.csv')
train.label.value_counts()

0    446477
1     12321
Name: label, dtype: int64

In [4]:
print(f"Dimensions of train data : {train.shape}")
print(f"Dimensions of test data : {test.shape}")

Dimensions of train data : (458798, 53)
Dimensions of test data : (24148, 52)


In [5]:
train = train.drop(columns = ['user_id'])
test = test.drop(columns = ['user_id'])
train.head()

,label,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X42,X43,X44,X45,X46,X47,X48,X49,X50,X51
0,0,20972,14,13,3,11,1,2,15,26,...,0,0,0,0,0,0,0,0,1,0
1,0,7362,213,71,3,11,1,2,15,26,...,0,0,0,0,0,0,0,0,0,0
2,0,21216,215,71,3,11,1,2,15,14,...,0,0,0,0,0,0,0,0,0,0
3,0,2261,212,71,2,8,1,2,15,14,...,0,0,0,0,0,0,0,1,0,0
4,0,4543,213,71,2,8,1,2,15,14,...,0,0,0,0,0,0,0,0,0,0


In [6]:
X = train.drop(columns = ['label'])
y = train[['label']]

In [7]:
params = { 'max_depth': [7],
           'learning_rate': [0.01, 0.1, 0.2, 0.3],
           'subsample': np.arange(0.5, 1.0, 0.1),
           'colsample_bytree': np.arange(0.4, 1.0, 0.1),
           'colsample_bylevel': np.arange(0.4, 1.0, 0.1),
           'n_estimators': [100, 500, 1000]}


In [8]:
model = xgb.XGBClassifier(max_depth = 10, learning_rate=0.2,n_estimators=250,subsample = 0.95,reg_alpha = 0.2, reg_lambda = 0.95, gamma = 0.1)

In [9]:
model.fit(X, y)
print(model)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0.1, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.2, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=10, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=250,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0.2, reg_lambda=0.95, ...)


In [10]:
predictions = model.predict(test)
np.unique(predictions, return_counts=True)

(array([0, 1]), array([23204,   944]))

In [11]:
predictions_train = model.predict(X)
print(metrics.classification_report(y, predictions_train))
print(metrics.confusion_matrix(y, predictions_train))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    446477
           1       0.98      0.89      0.93     12321

    accuracy                           1.00    458798
   macro avg       0.99      0.95      0.97    458798
weighted avg       1.00      1.00      1.00    458798

[[446209    268]
 [  1336  10985]]


In [12]:
submission = pd.read_csv('/kaggle/input/predict-potential-spammers-on-fiverr/sample_submission.csv')
submission.head()

,user_id,label
0,57,0
1,58,0
2,59,0
3,60,0
4,61,0


In [13]:
submission.label = predictions
submission.columns = ['user_id', 'prediction']

In [14]:
submission.to_csv('submission.csv', index = False)